In [ ]:
# Interactive AoA Network in Jupyter
# Enable interactive matplotlib backend for Jupyter
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
from datetime import date, timedelta
from collections import defaultdict
import json
from matplotlib.widgets import Button

# Import the project utilities
import sys
import os
from project_utils.project import Project

global project
print("Interactive matplotlib backend enabled for Jupyter")


## Enter tasks
When copying tables from PDFs to a text editor like Sublime, the table gets flattened and each cell forms a separate row. To disentangle the cells and extract column by column, use `awk 'NR % 4 == 1' <input file>` to extract the 1st column, use `4 == 2` to extract the second column etc. and use `4 == 0` to extract the last column.

In [ ]:
from datetime import date

project_name = 'iDesign Normal Solution Lab Example Project'
project_start_date = date(2020, 1, 6)

tasks = {
    'Task Name': ['Requirements', 'Architecture', 'Project Planning', 'Test Plan', 'Test Harness', 'Logging', 'Security', 'Pub/Sub', 'Resource A', 'Resource B', 'Resource Access A', 'Resource Access B', 'Resource Access C', 'EngineA', 'EngineB', 'EngineC', 'ManagerA', 'ManagerB', 'Client App1', 'Client App2', 'System Testing'],
    'Duration': [15, 20, 20, 30, 35, 15, 20, 5, 20, 15, 10, 5, 15, 20, 25, 15, 20, 25, 25, 35, 30],
    'Predecessors': ['', '1', '2', '3', '4', '3', '3', '3', '3', '3', '6,9', '6,10', '6', '12,13', '12,13', '6', '7,8,11,14,15', '7,8,15,16', '17,18', '17', '5,19,20']
}

assert len(tasks['Task Name']) == len(tasks['Duration']) == len(tasks['Predecessors'])

In [ ]:
# Interactive Task Entry using qgridnext (modern qgrid replacement)
import qgridnext as qgrid
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Create DataFrame
task_df = pd.DataFrame(tasks)

# Create qgrid widget for interactive editing
qgrid_widget = qgrid.show_grid(
    task_df, 
    show_toolbar=True,
    grid_options={
        'syncColumnCellResize': True,
        'forceFitColumns': False,
        'defaultColumnWidth': 150,
        'enableColumnReorder': False,
        'enableTextSelectionOnCells': True,
        'rowHeight': 28,
        'enableCellNavigation': True,
        'fullWidthRows': True
    },
    column_options={
        'editable': True
    },
    column_definitions={
        'Task Name': {'width': 200, 'editable': True},
        'Duration': {'width': 100, 'editable': True},
        'Predecessors': {'width': 150, 'editable': True}
    }
)

# Output area for messages and results
output_area = widgets.Output()

# Function to create project from qgrid data
def create_project_from_qgrid():
    """Create a project from the qgrid data and return it."""
    # Get the current data from qgrid
    current_data = qgrid_widget.get_changed_df()
    
    # Create new project
    project = Project(project_name, project_start_date)
    
    # Add tasks from the grid data
    for idx, row in current_data.iterrows():
        task_name = row['Task Name']
        duration = int(row['Duration'])
        predecessors_str = str(row['Predecessors']).strip()
        
        if predecessors_str and predecessors_str != 'nan':
            # Parse predecessors (comma-separated string)
            pred_list = [int(p.strip()) for p in predecessors_str.split(',') if p.strip().isdigit()]
            project.add_task(task_name, duration, predecessors=pred_list)
        else:
            project.add_task(task_name, duration)
    
    return project

# Function to add a new row to the qgrid
def add_new_task():
    """Add a new task row to the qgrid."""
    with output_area:
        output_area.clear_output()
        try:
            current_data = qgrid_widget.get_changed_df()
            next_task_num = len(current_data) + 1
            
            # Create new row
            new_row = pd.DataFrame({
                'Task Name': [f'Task {next_task_num}'],
                'Duration': [0],
                'Predecessors': ['']
            })
            
            # Add to existing data
            updated_data = pd.concat([current_data, new_row], ignore_index=True)
            
            # Update the qgrid with new data
            qgrid_widget.df = updated_data
            
            print(f"➕ Added Task {next_task_num}")
        except Exception as e:
            print(f"❌ Error adding task: {e}")

# Function to remove the last row from qgrid
def remove_last_task():
    """Remove the last task row from the qgrid."""
    with output_area:
        output_area.clear_output()
        try:
            current_data = qgrid_widget.get_changed_df()
            if len(current_data) <= 1:
                print("❌ Cannot remove the last task. At least one task is required.")
                return
            
            # Remove last row
            updated_data = current_data.iloc[:-1].copy()
            
            # Update the qgrid
            qgrid_widget.df = updated_data
            
            print(f"🗑️ Removed last task")
        except Exception as e:
            print(f"❌ Error removing task: {e}")

# Function to reset to original data
def reset_to_default():
    """Reset qgrid to original task data."""
    with output_area:
        output_area.clear_output()
        qgrid_widget.df = pd.DataFrame(tasks)
        print("🔄 Reset to default values")

# Function to preview current data
def preview_data():
    """Preview current qgrid data."""
    with output_area:
        output_area.clear_output()
        current_data = qgrid_widget.get_changed_df()
        print("📋 Current Task Data:")
        display(current_data)

# Event handler for apply changes
def on_apply_changes(b):
    with output_area:
        output_area.clear_output()
        try:
            global project
            project = create_project_from_qgrid()
            print(f"✅ Project updated: {project.name}")
            print(f"📊 Tasks: {len(project.tasks)}")
            
            # Display updated task schedule
            task_schedule = project.schedule_tasks()
            display(task_schedule)
        except Exception as e:
            print(f"❌ Error: {e}")

# Create buttons
add_task_button = widgets.Button(description="➕ Add New Task", button_style='primary')
remove_task_button = widgets.Button(description="🗑️ Remove Last Task", button_style='warning')
apply_button = widgets.Button(description="Apply Changes & Create Project", button_style='success')
reset_button = widgets.Button(description="Reset to Default", button_style='warning')
preview_button = widgets.Button(description="Preview Data", button_style='info')

# Connect event handlers
add_task_button.on_click(lambda b: add_new_task())
remove_task_button.on_click(lambda b: remove_last_task())
apply_button.on_click(on_apply_changes)
reset_button.on_click(lambda b: reset_to_default())
preview_button.on_click(lambda b: preview_data())

# Organize buttons
task_management_buttons = widgets.HBox([add_task_button, remove_task_button])
main_buttons = widgets.HBox([apply_button, reset_button, preview_button])

# Add a title
title = widgets.HTML(value="<h3>Interactive Task Editor (QGrid)</h3><p>Edit tasks in the grid below. Double-click cells to edit them.</p>")

on_apply_changes(apply_button)

# Display the interface
display(title)
display(qgrid_widget)
display(task_management_buttons)
display(main_buttons)
display(output_area)

print("🎯 QGrid task editor ready!")
print("📝 Features:")
print("  • Edit tasks by double-clicking cells in the grid")
print("  • Add new tasks with the '➕ Add New Task' button")
print("  • Remove the last task with the '🗑️ Remove Last Task' button")
print("  • Use toolbar buttons for sorting, filtering, and more")
print("  • Preview changes before applying")
print("  • Reset to original values anytime")
print("  • Click 'Apply Changes & Create Project' when ready!")


In [ ]:
# The following code is a backup approach should the interactive approach above not work.

# # Create and set up the project
# project = Project("iDesign Lab Example Project", date(2020, 1, 6))

# # Add tasks
# project.add_task("Task 1", 10)
# project.add_task("Task 2", 20, predecessors=[1])
# project.add_task("Task 3", 40, predecessors=[1])
# project.add_task("Task 4", 30, predecessors=[1])
# project.add_task("Task 5", 10, predecessors=[2])
# project.add_task("Task 6", 0, predecessors=[3])
# project.add_task("Task 7", 10, predecessors=[3])
# project.add_task("Task 8", 30, predecessors=[5])
# project.add_task("Task 9", 20, predecessors=[6, 8])
# project.add_task("Task 10", 25, predecessors=[6, 8])
# project.add_task("Task 11", 10, predecessors=[4, 7, 9])
# project.add_task("Task 12", 10, predecessors=[10])
# project.add_task("Task 13", 0, predecessors=[11, 12])
# project.add_task("Task 14", 10, predecessors=[11, 12])
# project.add_task("Task 15", 5, predecessors=[11, 12])
# project.add_task("Task 16", 5, predecessors=[13])
# project.add_task("Task 17", 5, predecessors=[14, 16])
# project.add_task("Task 18", 5, predecessors=[15, 17])

print(f"Project: {project.name}")
print(f"Start date: {project.start_date.strftime('%A, %B %d, %Y')}")
print(f"Tasks: {len(project.tasks)}")
task_df = project.schedule_tasks()
task_df


In [ ]:
# Generate the AoA network with interactive capability
def setup_interactive_aoa_plot():
    """Set up interactive AoA plotting in Jupyter."""
    
    # Generate the plot (this populates the aoa_*_artists attributes)
    g = project.draw_aoa_network_float_based(title=f"Interactive AoA Network - {project.name}")
    
    # Get current figure and axis
    fig = plt.gcf()
    ax = plt.gca()
    
    # Set up node tracking for interactivity
    node_order = getattr(project, 'aoa_node_order', list(g.nodes()))
    node_collection = None
    node_positions = {}
    node_indices = {}
    node_labels = {}  # Store references to node label text artists

    # Build correct mapping from nodes to their positions and indices
    for idx, node in enumerate(node_order):
        if node in project.aoa_node_artists:
            artist, pos = project.aoa_node_artists[node]
            node_collection = artist  # All nodes share the same PathCollection
            node_positions[node] = pos
            node_indices[node] = idx

    # Find and store node label text artists
    for text_artist in ax.texts:
        # Node labels have format like "E0" or similar
        if hasattr(text_artist, 'get_text'):
            text_content = text_artist.get_text()
            if text_content.startswith('E'):
                try:
                    node_num = int(text_content[1:])  # Remove 'E' and get number
                    if node_num in node_positions:
                        node_labels[node_num] = text_artist
                except (ValueError, IndexError):
                    pass

    edge_artists = getattr(project, 'aoa_edge_artists', {})
    label_artists = getattr(project, 'aoa_label_artists', {})

    dragging = {'node': None, 'offset': (0, 0)}

    def hit_test(event):
        if event.inaxes != ax or event.xdata is None or event.ydata is None:
            return None
        
        tol = 0.8  # Tolerance for clicking
        closest_node = None
        closest_distance = float('inf')
        
        # Find the closest node within tolerance
        for node, (x, y) in node_positions.items():
            distance = np.hypot(event.xdata - x, event.ydata - y)
            if distance < tol and distance < closest_distance:
                closest_distance = distance
                closest_node = node
        
        return closest_node

    def on_press(event):
        node = hit_test(event)
        if node is not None:
            dragging['node'] = node
            x, y = node_positions[node]
            dragging['offset'] = (event.xdata - x, event.ydata - y)
            print(f"Grabbed node E{node} at position ({x:.2f}, {y:.2f})")

    def on_motion(event):
        node = dragging['node']
        if node is not None and event.inaxes == ax and event.xdata is not None and event.ydata is not None:
            new_x = event.xdata - dragging['offset'][0]
            new_y = event.ydata - dragging['offset'][1]
            node_positions[node] = (new_x, new_y)
            
            # Update PathCollection offsets
            if node in node_indices and node_collection is not None:
                idx = node_indices[node]
                offsets = node_collection.get_offsets()
                offsets[idx] = [new_x, new_y]
                node_collection.set_offsets(offsets)
            
            # Update node label position
            if node in node_labels:
                node_labels[node].set_position((new_x, new_y))
            
            # Update connected edges and labels
            for (u, v), edge_artist in edge_artists.items():
                if u == node or v == node:
                    x1, y1 = node_positions[u]
                    x2, y2 = node_positions[v]
                    # Handle FancyArrowPatch objects
                    if hasattr(edge_artist, 'set_positions'):
                        edge_artist.set_positions((x1, y1), (x2, y2))
                    elif hasattr(edge_artist, 'set_data'):
                        edge_artist.set_data([x1, x2], [y1, y2])
                    else:
                        # For FancyArrowPatch, update the path
                        edge_artist.set_position_a((x1, y1))
                        edge_artist.set_position_b((x2, y2))
            
            for (u, v), label_artist in label_artists.items():
                if u == node or v == node:
                    x1, y1 = node_positions[u]
                    x2, y2 = node_positions[v]
                    label_x = (x1 + x2) / 2
                    label_y = (y1 + y2) / 2
                    label_artist.set_position((label_x, label_y))
            
            fig.canvas.draw_idle()

    def on_release(event):
        node = dragging['node']
        if node is not None:
            print(f"Released node E{node}")
            dragging['node'] = None

    def save_layout(event=None):
        """Save current node positions to JSON file."""
        layout = {int(node): {'x': float(pos[0]), 'y': float(pos[1])} 
                 for node, pos in node_positions.items()}
        with open('aoa_layout.json', 'w') as f:
            json.dump(layout, f, indent=2)
        print("Layout saved to aoa_layout.json")

    def on_key(event):
        """Handle keyboard shortcuts."""
        if event.key == 's':
            save_layout()

    # Connect event handlers
    fig.canvas.mpl_connect('button_press_event', on_press)
    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    fig.canvas.mpl_connect('button_release_event', on_release)
    fig.canvas.mpl_connect('key_press_event', on_key)

    # Add save button
    button_ax = fig.add_axes([0.85, 0.01, 0.12, 0.05])
    save_button = Button(button_ax, 'Save Layout')
    save_button.on_clicked(save_layout)

    print('Interactive AoA editing enabled in Jupyter!')
    print('- Drag nodes to move them')
    print('- Press "s" to save layout')
    print('- Click "Save Layout" button to save')
    
    return fig, ax

# Call the setup function
fig, ax = setup_interactive_aoa_plot()


## Staffing requirements
- A project manager, a product owner and an architect form the core team and is available throughout the entire project.
- Each service/task requires a single person. No two persons ever work on the same service/task.
- For the first project design iteration assume that every person will be available on demand, even though in reality staffing is not that elastic.
- A person can leave the project, but not join again to avoid on-and-off scenarios.
- The initial project design iteration can use unlimited resources.
- For specialist tasks like database design start with the assumption that enough specialists available (e.g. DBAs) and create variants of the project design by substituting specialists by generalists (i.e. developers) one by one, assuming that the outcome created by generalists is usually not as well designed as by a specialist and might take longer. Compare the variants to see whether it makes a significant difference.
- Assign the best developer to the critical path, the second best to the second-most critical etc.
- ... (add your specific staffing requirements)

In [ ]:
# Mapping of resource types to staffing phases
# ============================================

# To do:
# - adapt the staffing phases to your project
# - adapt the mapping of resource types to staffing phases to your project

# Notes:
# - DevOps is also known as Configuration Manager (CM)
# - Not all staffing resources are required at all phases of the project.

unlimited_staffing_phases = {
    # the fuzzy project front-end
    'Planning': {'Architect', 'Project Manager', 'Product Owner'},
    # foundational work, i.e. non-business logic services and utilities like logging, security, pub/sub etc.
    'Infrastructure': {'Architect', 'Project Manager', 'Product Owner', 'Developer', 'DevOps'},
    # the business logic services
    'Services': {'Architect', 'Project Manager', 'Product Owner', 'Developer', 'DevOps Engineer', 'Tester'},
    # verification and validation
    'Testing': {'Architect', 'Project Manager', 'Product Owner', 'DevOps Engineer', 'Tester'},
}

# The following resource name stems are used to create unique resource names.
resource_name_stems = {
    'Architect': 'Arch',
    'Project Manager': 'PM',
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',
    'DB Architect': 'DBA',
    'Quality Control Tester': 'QC',
}


# Mapping of tasks to resource types
# ==================================

# To do:
# - adapt the mapping of tasks to resource types to your project

# len(tasks['Task Name'])
who_does_what = {
    'Requirements': 'Architect',
    'Architecture': 'Architect',
    'Project Planning': 'Architect',
    'Test Plan': 'Test Engineer',
    'Test Harness': 'Test Engineer',
    'Logging': 'Developer',
    'Security': 'Developer',
    'Pub/Sub': 'Developer',
    'Resource A': 'DB Architect',
    'Resource B': 'DB Architect',
    'Resource Access A': 'Developer',
    'Resource Access B': 'Developer',
    'Resource Access C': 'Developer',
    'EngineA': 'Developer',
    'EngineB': 'Developer',
    'EngineC': 'Developer',
    'ManagerA': 'Developer',
    'ManagerB': 'Developer',
    'Client App1': 'Developer',
    'Client App2': 'Developer',
    'System Testing': 'Quality Control Tester',
}

assert len(tasks['Task Name']) == len(who_does_what)

In [ ]:
# Planned Earned Value Analysis Using Project Class Method
# ========================================================

print("Updated Planned Earned Value Analysis (Using Project Class Method)")
print("=" * 70)

# Get the scheduled task data and assignments
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

# Get task assignments using the resource mapping from above
person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

# Calculate planned earned value using the NEW Project class method
earned_value_data = project.calculate_planned_earned_value(scheduled_tasks, person_assignments)

# Extract data from the result
total_duration = earned_value_data['total_duration']
enhanced_tasks = earned_value_data['sorted_tasks']
summary_stats = earned_value_data['summary_stats']

print(f"📊 Total Project Duration: {total_duration} days")

# Display the planned earned value table
print(f"\n📋 Planned Earned Value Analysis:")
print("=" * 80)
print(f"{'Task ID':<10} {'Owner':<12} {'Task Name':<25} {'Duration':<10} {'Completion':<12} {'Earned Value':<12}")
print("-" * 80)

for task_data in enhanced_tasks:
    # Format the output
    short_task_name = task_data['name'][:23] + '..' if len(task_data['name']) > 25 else task_data['name']
    short_owner = task_data['owner'][:10] + '..' if len(task_data['owner']) > 12 else task_data['owner']
    completion_str = str(task_data['completion_date'])[:10] if task_data['completion_date'] != 'N/A' else 'N/A'
    
    print(f"{task_data['task_id']:<10} {short_owner:<12} {short_task_name:<25} {task_data['duration']:<10} {completion_str:<12} {task_data['earned_value']:>10.1f}%")

print("-" * 80)
print(f"📈 Final Earned Value: 100.0%")
print(f"📅 Project Completion: {summary_stats['project_completion_date']}")

# Display summary statistics
print(f"\n📊 Summary Statistics:")
print(f"   • Total Tasks: {summary_stats['total_tasks']}")
print(f"   • Critical Path Tasks: {summary_stats['critical_path_tasks']}")
print(f"   • Average Task Duration: {summary_stats['average_task_duration']:.1f} days")
print(f"   • Total People Assigned: {summary_stats['total_people_assigned']}")


In [ ]:
# Planned Earned Value Chart: Time vs. Percent (Excel Line with Markers Style)
# ============================================================================

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd

print("Creating Planned Earned Value Chart")
print("=" * 50)

# Get the earned value data from the previous cell's calculation
# (We'll recalculate to ensure we have the data available)
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

earned_value_data = project.calculate_planned_earned_value(scheduled_tasks, person_assignments)
enhanced_tasks = earned_value_data['sorted_tasks']
total_duration = earned_value_data['total_duration']

# Prepare data for plotting
dates = []
earned_values = []
task_names = []

for task_data in enhanced_tasks:
    completion_date = task_data['completion_date']
    
    # Handle date conversion
    if completion_date != 'N/A' and completion_date is not None:
        try:
            # Try to convert to datetime if it's not already
            if isinstance(completion_date, str):
                # Assuming date format like '2024-01-15' or similar
                date_obj = pd.to_datetime(completion_date).date()
            else:
                date_obj = completion_date
            
            dates.append(date_obj)
            earned_values.append(task_data['earned_value'])
            task_names.append(task_data['name'][:20] + '..' if len(task_data['name']) > 20 else task_data['name'])
        except:
            # Skip tasks with invalid dates
            continue

# Create the plot in Excel "line with markers" style
plt.figure(figsize=(14, 8))

# Plot the line with markers (Excel style)
plt.plot(dates, earned_values, 
         marker='o',           # Circle markers
         markersize=6,         # Marker size
         markerfacecolor='#4472C4',  # Excel blue
         markeredgecolor='#4472C4',
         markeredgewidth=1,
         linewidth=2.5,        # Line thickness
         color='#4472C4',      # Excel blue color
         linestyle='-',        # Solid line
         alpha=0.9)

# Excel-style formatting
plt.grid(True, linestyle=':', alpha=0.6, color='gray')
plt.gca().set_facecolor('#FFFFFF')  # White background
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('#CCCCCC')
plt.gca().spines['bottom'].set_color('#CCCCCC')

# Formatting axes
plt.xlabel('Project Timeline (Completion Dates)', fontsize=12, fontweight='bold')
plt.ylabel('Planned Earned Value (%)', fontsize=12, fontweight='bold')
plt.title('Planned Earned Value Over Time\n(Cumulative Progress by Task Completion)', 
          fontsize=14, fontweight='bold', pad=20)

# Format y-axis to show percentages
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))
plt.ylim(0, 105)  # Start from 0% and give a bit of space above 100%

# Format x-axis for dates
if dates:
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    plt.xticks(rotation=45, ha='right')

# Tight layout and display
plt.tight_layout()
plt.show()

# Print summary information
print(f"\n📊 Chart Summary:")
print(f"   • Total data points plotted: {len(dates)}")
print(f"   • Project duration: {total_duration} days")
print(f"   • Start date: {min(dates) if dates else 'N/A'}")
print(f"   • End date: {max(dates) if dates else 'N/A'}")
print(f"   • Final earned value: {max(earned_values) if earned_values else 0:.1f}%")

# Display first few and last few data points
if len(dates) > 0:
    print(f"\n📅 Sample Data Points:")
    print(f"{'Date':<12} {'Earned Value':<12} {'Task':<25}")
    print("-" * 50)
    
    # Show first 3 points
    for i in range(min(3, len(dates))):
        print(f"{dates[i]:<12} {earned_values[i]:>10.1f}% {task_names[i]:<25}")
    
    if len(dates) > 6:
        print("   ...")
        # Show last 3 points
        for i in range(max(0, len(dates)-3), len(dates)):
            print(f"{dates[i]:<12} {earned_values[i]:>10.1f}% {task_names[i]:<25}")


In [ ]:
# UPDATED: Planned Earned Value Analysis Using Project Class Method
# ===============================================================

print("Updated Planned Earned Value Analysis (Using Project Class Method)")
print("=" * 70)

# Get the scheduled task data and assignments
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

# Get task assignments using the resource mapping from above
person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

# Calculate planned earned value using the NEW Project class method
earned_value_data = project.calculate_planned_earned_value(scheduled_tasks, person_assignments)

# Extract data from the result
total_duration = earned_value_data['total_duration']
enhanced_tasks = earned_value_data['sorted_tasks']
summary_stats = earned_value_data['summary_stats']

print(f"📊 Total Project Duration: {total_duration} days")

# Display the planned earned value table
print(f"\n📋 Planned Earned Value Analysis:")
print("=" * 80)
print(f"{'Task ID':<10} {'Owner':<12} {'Task Name':<25} {'Duration':<10} {'Completion':<12} {'Earned Value':<12}")
print("-" * 80)

for task_data in enhanced_tasks:
    # Format the output
    short_task_name = task_data['name'][:23] + '..' if len(task_data['name']) > 25 else task_data['name']
    short_owner = task_data['owner'][:10] + '..' if len(task_data['owner']) > 12 else task_data['owner']
    completion_str = str(task_data['completion_date'])[:10] if task_data['completion_date'] != 'N/A' else 'N/A'
    
    print(f"{task_data['task_id']:<10} {short_owner:<12} {short_task_name:<25} {task_data['duration']:<10} {completion_str:<12} {task_data['earned_value']:>10.1f}%")

print("-" * 80)
print(f"📈 Final Earned Value: 100.0%")
print(f"📅 Project Completion: {summary_stats['project_completion_date']}")

# Display summary statistics
print(f"\n📊 Summary Statistics:")
print(f"   • Total Tasks: {summary_stats['total_tasks']}")
print(f"   • Critical Path Tasks: {summary_stats['critical_path_tasks']}")
print(f"   • Average Task Duration: {summary_stats['average_task_duration']:.1f} days")
print(f"   • Total People Assigned: {summary_stats['total_people_assigned']}")


In [ ]:
# Testing Assignment Validation: validate_assignments method
# ========================================================

print("Testing validate_assignments Method")
print("=" * 50)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

# Resource mapping
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',
    'DBA': 'DBA',
    'Quality Control Tester': 'QC',
    'Tester': 'Test'
}

print(f"📊 Project: {project.name}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# SCENARIO 1: Test with VALID assignments (from our main algorithm)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 1: Testing with VALID assignments")
print("=" * 80)

print("Running assignment algorithm to get valid assignments...")
try:
    valid_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated,
        max_gap_days=7
    )
    
    print(f"\n✅ Assignment algorithm completed successfully!")
    print(f"Now validating these assignments...")
    
    # Validate the assignments
    is_valid = project.validate_assignments(valid_assignments, scheduled_tasks)
    
    print(f"\n🎯 SCENARIO 1 RESULT: {'✅ PASSED' if is_valid else '❌ FAILED'}")
    
except Exception as e:
    print(f"❌ Assignment algorithm failed: {e}")
    valid_assignments = {}

# SCENARIO 2: Test with INVALID assignments (intentional overlaps)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 2: Testing with INVALID assignments (overlapping tasks)")
print("=" * 80)

print("Creating intentionally invalid assignments with overlapping tasks...")

# Helper function to find a task by name
def find_task_by_name(task_name):
    for task in scheduled_tasks:
        if task['Name'] == task_name:
            return task
    return None

# Create invalid assignments with overlapping tasks
invalid_assignments = {}

# Assign overlapping tasks to same person (this should fail validation)
developer_1_tasks = []
logging_task = find_task_by_name('Logging')
security_task = find_task_by_name('Security')
pubsub_task = find_task_by_name('Pub/Sub')

if logging_task and security_task and pubsub_task:
    # These tasks likely overlap (they all depend on task 3)
    developer_1_tasks = [logging_task, security_task, pubsub_task]
    invalid_assignments['Dev_1'] = developer_1_tasks
    
    print(f"Assigned overlapping tasks to Dev_1:")
    for task in developer_1_tasks:
        print(f"  • {task['Name']} ({task['Start']} - {task['Finish']})")

# Add some other assignments to make it more realistic
architect_task = find_task_by_name('Requirements')
if architect_task:
    invalid_assignments['Arch_1'] = [architect_task]

# Validate the invalid assignments
print(f"\nNow validating these INTENTIONALLY INVALID assignments...")
is_valid_2 = project.validate_assignments(invalid_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 2 RESULT: {'❌ FAILED as expected' if not is_valid_2 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 3: Test with INCOMPLETE assignments (missing tasks)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 3: Testing with INCOMPLETE assignments (missing tasks)")
print("=" * 80)

print("Creating incomplete assignments (not all tasks assigned)...")

# Create assignments with only some tasks
incomplete_assignments = {}

# Only assign first few tasks
first_few_tasks = scheduled_tasks[:5]  # Only first 5 tasks
for i, task in enumerate(first_few_tasks):
    person_name = f"Person_{i+1}"
    incomplete_assignments[person_name] = [task]
    print(f"  • {person_name}: {task['Name']}")

print(f"\nOnly assigned {len(first_few_tasks)} out of {len(scheduled_tasks)} tasks")

# Validate the incomplete assignments  
print(f"\nNow validating these INCOMPLETE assignments...")
is_valid_3 = project.validate_assignments(incomplete_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 3 RESULT: {'❌ FAILED as expected' if not is_valid_3 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 4: Test with DUPLICATE assignments (same task assigned twice)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 4: Testing with DUPLICATE assignments (same task assigned twice)")
print("=" * 80)

print("Creating assignments with duplicate task assignments...")

# Create assignments where same task is assigned to multiple people
duplicate_assignments = {}

requirements_task = find_task_by_name('Requirements')
if requirements_task:
    # Assign same task to two different people
    duplicate_assignments['Person_A'] = [requirements_task]
    duplicate_assignments['Person_B'] = [requirements_task]  # Same task!
    
    print(f"Assigned '{requirements_task['Name']}' to both Person_A and Person_B")

# Add a few more normal assignments
for i, task in enumerate(scheduled_tasks[1:4]):  # Tasks 2, 3, 4
    person_name = f"Person_{i+3}"
    duplicate_assignments[person_name] = [task]

# Validate the duplicate assignments
print(f"\nNow validating these assignments with DUPLICATES...")
is_valid_4 = project.validate_assignments(duplicate_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 4 RESULT: {'❌ FAILED as expected' if not is_valid_4 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 5: Test with EMPTY assignments
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 5: Testing with EMPTY assignments")
print("=" * 80)

print("Testing validation with empty assignments...")

empty_assignments = {}

# Validate empty assignments
is_valid_5 = project.validate_assignments(empty_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 5 RESULT: {'❌ FAILED as expected' if not is_valid_5 else '⚠️ UNEXPECTEDLY PASSED'}")

# SUMMARY OF ALL TEST SCENARIOS
print(f"\n" + "=" * 80)
print("🎯 VALIDATION TESTING SUMMARY")
print("=" * 80)

scenarios = [
    ("VALID assignments", is_valid if 'is_valid' in locals() else False, True),
    ("OVERLAPPING assignments", is_valid_2, False),
    ("INCOMPLETE assignments", is_valid_3, False),
    ("DUPLICATE assignments", is_valid_4, False),
    ("EMPTY assignments", is_valid_5, False)
]

print(f"\n📊 Test Results:")
print(f"{'Scenario':<25} {'Result':<10} {'Expected':<10} {'Status'}")
print("-" * 60)

all_tests_passed = True

for i, (scenario, actual, expected) in enumerate(scenarios, 1):
    status = "✅ PASS" if actual == expected else "❌ FAIL"
    if actual != expected:
        all_tests_passed = False
    
    actual_str = "VALID" if actual else "INVALID"
    expected_str = "VALID" if expected else "INVALID"
    
    print(f"{scenario:<25} {actual_str:<10} {expected_str:<10} {status}")

print(f"\n🎯 OVERALL VALIDATION TESTING: {'✅ ALL TESTS PASSED' if all_tests_passed else '❌ SOME TESTS FAILED'}")

if all_tests_passed:
    print("🎉 The validate_assignments method correctly identifies:")
    print("   • Valid assignments with no overlaps")
    print("   • Invalid assignments with overlapping tasks")
    print("   • Incomplete assignments with missing tasks")
    print("   • Invalid assignments with duplicate tasks")
    print("   • Empty assignments")
    print("\n✨ Assignment validation is working perfectly!")
else:
    print("⚠️  Some validation tests failed. Check the implementation.")

print("\n📝 The validation method provides:")
print("   • Detailed overlap checking per person")
print("   • Complete task assignment verification")
print("   • Comprehensive assignment summaries")
print("   • Clear pass/fail validation results")
print("   • Detailed error reporting and warnings")
print("   • Workload balance analysis")


In [ ]:
# Testing the tasks_overlap method
# ===============================

print("Testing tasks_overlap method")
print("=" * 40)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()

# Convert to list of dictionaries for easier access
scheduled_tasks = task_schedule.to_dict('records')

# Helper function to find a task by name
def find_task_by_name(task_name):
    for task in scheduled_tasks:
        if task['Name'] == task_name:
            return task
    return None

# Test Case 1: Tasks that overlap in time
print("\n1. Testing OVERLAPPING tasks:")
print("-" * 30)

# Get some tasks that should overlap (tasks that can run in parallel)
task1 = find_task_by_name('Logging')
task2 = find_task_by_name('Security')

if task1 and task2:
    print(f"Task 1: {task1['Name']}")
    print(f"  Start: {task1['Start']}")
    print(f"  Finish: {task1['Finish']}")
    print(f"Task 2: {task2['Name']}")
    print(f"  Start: {task2['Start']}")
    print(f"  Finish: {task2['Finish']}")
    
    overlap_result = project.tasks_overlap(task1, task2)
    print(f"Tasks overlap? {overlap_result}")
    print()

# Test Case 2: Tasks that do NOT overlap (sequential tasks)
print("2. Testing NON-OVERLAPPING tasks:")
print("-" * 35)

task3 = find_task_by_name('Requirements')
task4 = find_task_by_name('Architecture')

if task3 and task4:
    print(f"Task 1: {task3['Name']}")
    print(f"  Start: {task3['Start']}")
    print(f"  Finish: {task3['Finish']}")
    print(f"Task 2: {task4['Name']}")
    print(f"  Start: {task4['Start']}")
    print(f"  Finish: {task4['Finish']}")
    
    overlap_result = project.tasks_overlap(task3, task4)
    print(f"Tasks overlap? {overlap_result}")
    print()

# Test Case 3: Edge case - Adjacent tasks (one ends when the other starts)
print("3. Testing ADJACENT tasks (edge case):")
print("-" * 38)

# Find tasks where one immediately follows another
task5 = find_task_by_name('Architecture')
task6 = find_task_by_name('Project Planning')

if task5 and task6:
    print(f"Task 1: {task5['Name']}")
    print(f"  Start: {task5['Start']}")
    print(f"  Finish: {task5['Finish']}")
    print(f"Task 2: {task6['Name']}")
    print(f"  Start: {task6['Start']}")
    print(f"  Finish: {task6['Finish']}")
    
    overlap_result = project.tasks_overlap(task5, task6)
    print(f"Tasks overlap? {overlap_result}")
    print("Note: Adjacent tasks (where one ends when the other starts) are considered non-overlapping")
    print()

# Test Case 4: Multiple parallel tasks
print("4. Testing MULTIPLE PARALLEL tasks:")
print("-" * 35)

parallel_tasks = ['Logging', 'Security', 'Pub/Sub', 'Resource A', 'Resource B']
parallel_task_objects = [find_task_by_name(name) for name in parallel_tasks if find_task_by_name(name)]

print("Checking overlap between all parallel tasks that start after Project Planning:")
for i, task_i in enumerate(parallel_task_objects):
    for j, task_j in enumerate(parallel_task_objects):
        if i < j:  # Only test each pair once
            overlap_result = project.tasks_overlap(task_i, task_j)
            print(f"  {task_i['Name']} & {task_j['Name']}: {'OVERLAP' if overlap_result else 'NO OVERLAP'}")

print()

# Test Case 5: Error handling
print("5. Testing ERROR HANDLING:")
print("-" * 25)

# Test with tasks missing required fields
incomplete_task1 = {'Name': 'Incomplete Task 1', 'Duration': 10}
incomplete_task2 = {'Name': 'Incomplete Task 2', 'Duration': 15}

try:
    result = project.tasks_overlap(incomplete_task1, incomplete_task2)
    print("ERROR: Should have raised KeyError!")
except KeyError as e:
    print(f"✓ Correctly caught KeyError: {e}")

print("\n" + "=" * 50)
print("All tests completed successfully!")

# Summary table of a few key task pairs
print("\nSUMMARY - Task Overlap Analysis:")
print("-" * 35)
test_pairs = [
    ('Requirements', 'Architecture'),
    ('Logging', 'Security'),
    ('Architecture', 'Project Planning'),
    ('Resource A', 'Resource B'),
]

for task1_name, task2_name in test_pairs:
    t1 = find_task_by_name(task1_name)
    t2 = find_task_by_name(task2_name)
    if t1 and t2:
        overlap = project.tasks_overlap(t1, t2)
        status = "OVERLAPPING" if overlap else "SEQUENTIAL"
        print(f"{task1_name:15} & {task2_name:15}: {status}")


In [ ]:
# Testing No-Rejoining Constraint: Updated assign_tasks_to_persons
# ===============================================================

print("Testing Updated assign_tasks_to_persons with No-Rejoining Constraint")
print("=" * 70)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

print(f"📊 Project: {project.name}")
print(f"📅 Start Date: {project.start_date}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# Display critical path information
critical_tasks = [task for task in scheduled_tasks if task.get('TF', float('inf')) == 0]
print(f"🔥 Critical Path Tasks: {len(critical_tasks)}")

# Use the resource mapping from the earlier cell
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',
    'DBA': 'DBA',
    'Quality Control Tester': 'QC',
    'Tester': 'Test'
}

print(f"\n🚫 NO-REJOINING CONSTRAINT:")
print("   Once a person finishes work and has a gap > max_gap_days,")
print("   they leave the project and cannot be reassigned later.")
print("   This prevents on-and-off scenarios in staffing.")

# Test with different gap thresholds
gap_scenarios = [
    (3, "Strict (3 days)"),
    (7, "Moderate (7 days)"), 
    (14, "Lenient (14 days)")
]

results_by_scenario = {}

for max_gap, scenario_name in gap_scenarios:
    print(f"\n" + "=" * 80)
    print(f"🧪 TESTING SCENARIO: {scenario_name} - Max Gap: {max_gap} days")
    print("=" * 80)
    
    try:
        person_assignments = project.assign_tasks_to_persons(
            tasks=scheduled_tasks,
            task_resource_mapping=who_does_what,
            resource_name_stems=resource_name_stems_updated,
            max_gap_days=max_gap
        )
        
        # Collect results for comparison
        total_people = sum(len(people) for people in {})  # Will be calculated below
        active_people = len(person_assignments)
        
        # Count people by extracting from person names
        all_people = set()
        people_by_resource = {}
        left_people = set()
        
        for person_name in person_assignments.keys():
            all_people.add(person_name)
            resource_stem = person_name.split('_')[0]
            resource_type = None
            for full_type, stem in resource_name_stems_updated.items():
                if stem == resource_stem:
                    resource_type = full_type
                    break
            if resource_type:
                if resource_type not in people_by_resource:
                    people_by_resource[resource_type] = []
                people_by_resource[resource_type].append(person_name)
        
        # Note: We can't easily extract who left from the return value,
        # but the method prints this information during execution
        
        results_by_scenario[scenario_name] = {
            'max_gap': max_gap,
            'active_people': active_people,
            'people_by_resource': people_by_resource,
            'total_assignments': sum(len(tasks) for tasks in person_assignments.values())
        }
        
        print(f"\n✅ {scenario_name} scenario completed successfully!")
        print(f"   Active people: {active_people}")
        
    except Exception as e:
        print(f"\n❌ {scenario_name} scenario failed: {e}")
        results_by_scenario[scenario_name] = {'error': str(e)}

# Comparative analysis
print(f"\n" + "=" * 80)
print("📊 COMPARATIVE ANALYSIS - Impact of Gap Tolerance")
print("=" * 80)

print(f"\n{'Scenario':<20} {'Max Gap':<10} {'Active People':<15} {'Total Assignments'}")
print("-" * 70)

for scenario_name, results in results_by_scenario.items():
    if 'error' not in results:
        print(f"{scenario_name:<20} {results['max_gap']:<10} {results['active_people']:<15} {results['total_assignments']}")
    else:
        print(f"{scenario_name:<20} {'ERROR':<10} {'N/A':<15} {'N/A'}")

print(f"\n🔍 Key Insights:")
print("• Stricter gap policies (lower max_gap_days) may force more people to leave")
print("• This could result in creating more people overall for the same work")
print("• Lenient policies allow more flexibility but may create longer gaps")
print("• The optimal gap depends on project management preferences")

# Test edge cases
print(f"\n" + "=" * 80)
print("🧪 TESTING EDGE CASES")
print("=" * 80)

print(f"\n1. Testing with max_gap_days = 0 (immediate leaving):")
try:
    immediate_leaving = project.assign_tasks_to_persons(
        tasks=scheduled_tasks[:5],  # Test with fewer tasks
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated,
        max_gap_days=0
    )
    print("✅ Immediate leaving test completed")
except Exception as e:
    print(f"❌ Immediate leaving test failed: {e}")

print(f"\n2. Testing with max_gap_days = 365 (very lenient):")
try:
    lenient_leaving = project.assign_tasks_to_persons(
        tasks=scheduled_tasks[:5],  # Test with fewer tasks
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated,
        max_gap_days=365
    )
    print("✅ Lenient leaving test completed")
except Exception as e:
    print(f"❌ Lenient leaving test failed: {e}")

print(f"\n🎯 No-Rejoining Constraint Testing Completed!")
print("The constraint successfully prevents people from rejoining the project")
print("once they have left due to work gaps, eliminating on-and-off scenarios.")


In [ ]:
# Testing Person Assignment and Availability System
# ================================================

print("Testing Person Assignment and Availability System")
print("=" * 55)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

# Helper function to find a task by name
def find_task_by_name(task_name):
    for task in scheduled_tasks:
        if task['Name'] == task_name:
            return task
    return None

# Create sample person assignments dictionary
# Structure: {person_name: [list_of_assigned_task_objects]}
person_assignments = {}

print("\n📋 Setting up sample person assignments...")
print("-" * 45)

# Sample assignments based on the who_does_what mapping from earlier
# Let's assign some tasks to specific people

# Architect assignments
architect_tasks = ['Requirements', 'Architecture', 'Project Planning']
person_assignments['Alice_Architect'] = []
for task_name in architect_tasks:
    task = find_task_by_name(task_name)
    if task:
        person_assignments['Alice_Architect'].append(task)
        print(f"  Alice_Architect assigned to: {task_name} ({task['Start']} - {task['Finish']})")

# Developer assignments (parallel tasks)
developer_tasks = ['Logging', 'Security']
person_assignments['Bob_Developer'] = []
for task_name in developer_tasks:
    task = find_task_by_name(task_name)
    if task:
        person_assignments['Bob_Developer'].append(task)
        print(f"  Bob_Developer assigned to: {task_name} ({task['Start']} - {task['Finish']})")

# Another developer with different tasks
person_assignments['Carol_Developer'] = []
dev2_tasks = ['Pub/Sub']
for task_name in dev2_tasks:
    task = find_task_by_name(task_name)
    if task:
        person_assignments['Carol_Developer'].append(task)
        print(f"  Carol_Developer assigned to: {task_name} ({task['Start']} - {task['Finish']})")

# Test unassigned person
person_assignments['David_Developer'] = []
print(f"  David_Developer assigned to: (no tasks)")

print(f"\nTotal people tracked: {len(person_assignments)}")

# TEST CASE 1: Check availability for person with no assignments
print("\n" + "="*60)
print("TEST CASE 1: Person with no current assignments")
print("="*60)

new_task = find_task_by_name('Resource A')
if new_task:
    available = project.is_person_available('David_Developer', new_task, person_assignments)
    print(f"Person: David_Developer")
    print(f"New Task: {new_task['Name']} ({new_task['Start']} - {new_task['Finish']})")
    print(f"Currently assigned tasks: {len(person_assignments['David_Developer'])}")
    print(f"Is available? {available}")
    print("✓ Expected: True (no conflicts)")

# TEST CASE 2: Check availability for person with non-conflicting tasks
print("\n" + "="*60)
print("TEST CASE 2: Person with non-conflicting assignments")
print("="*60)

# Try to assign a task that doesn't overlap with Bob's current assignments
new_task = find_task_by_name('Resource A')  # This should start after Security/Logging
if new_task:
    available = project.is_person_available('Bob_Developer', new_task, person_assignments)
    print(f"Person: Bob_Developer")
    print(f"Current assignments:")
    for task in person_assignments['Bob_Developer']:
        print(f"  - {task['Name']} ({task['Start']} - {task['Finish']})")
    print(f"New Task: {new_task['Name']} ({new_task['Start']} - {new_task['Finish']})")
    print(f"Is available? {available}")
    print(f"✓ Expected: {available} (depends on actual scheduling)")

# TEST CASE 3: Check availability for person with conflicting tasks
print("\n" + "="*60)
print("TEST CASE 3: Person with conflicting assignments")
print("="*60)

# Try to assign Bob another task that overlaps with his current ones
new_task = find_task_by_name('Pub/Sub')  # This should overlap with Security/Logging
if new_task:
    available = project.is_person_available('Bob_Developer', new_task, person_assignments)
    print(f"Person: Bob_Developer")
    print(f"Current assignments:")
    for task in person_assignments['Bob_Developer']:
        print(f"  - {task['Name']} ({task['Start']} - {task['Finish']})")
    print(f"New Task: {new_task['Name']} ({new_task['Start']} - {new_task['Finish']})")
    print(f"Is available? {available}")
    
    # Show detailed overlap analysis
    print(f"\nDetailed overlap analysis:")
    for assigned_task in person_assignments['Bob_Developer']:
        overlap = project.tasks_overlap(new_task, assigned_task)
        print(f"  {new_task['Name']} overlaps with {assigned_task['Name']}: {overlap}")
    print(f"✓ Expected: False (has conflicts)")

# TEST CASE 4: Sequential task assignment (should be available)
print("\n" + "="*60)
print("TEST CASE 4: Sequential task assignment")
print("="*60)

# Try to assign Alice a task that comes after her current ones
new_task = find_task_by_name('Test Plan')  # Should come after Project Planning
if new_task:
    available = project.is_person_available('Alice_Architect', new_task, person_assignments)
    print(f"Person: Alice_Architect")
    print(f"Current assignments:")
    for task in person_assignments['Alice_Architect']:
        print(f"  - {task['Name']} ({task['Start']} - {task['Finish']})")
    print(f"New Task: {new_task['Name']} ({new_task['Start']} - {new_task['Finish']})")
    print(f"Is available? {available}")
    print(f"✓ Expected: {available} (should be available for sequential work)")

# TEST CASE 5: Error handling
print("\n" + "="*60)
print("TEST CASE 5: Error handling")
print("="*60)

# Test with incomplete task data
incomplete_task = {'Name': 'Incomplete Task', 'Duration': 10}
try:
    available = project.is_person_available('David_Developer', incomplete_task, person_assignments)
    print("ERROR: Should have raised KeyError!")
except KeyError as e:
    print(f"✓ Correctly caught KeyError: {e}")

# TEST CASE 6: Multiple assignment simulation
print("\n" + "="*60)
print("TEST CASE 6: Assignment simulation workflow")
print("="*60)

print("Simulating task assignment workflow:")

# List of tasks to assign
tasks_to_assign = ['Resource A', 'Resource B', 'EngineA', 'EngineB']
available_people = ['David_Developer', 'Eve_Developer', 'Frank_Developer']

# Initialize new people in assignments
for person in available_people:
    if person not in person_assignments:
        person_assignments[person] = []

assignment_results = []

for task_name in tasks_to_assign:
    task = find_task_by_name(task_name)
    if not task:
        continue
        
    print(f"\n🔍 Trying to assign: {task_name} ({task['Start']} - {task['Finish']})")
    
    assigned = False
    for person in available_people:
        available = project.is_person_available(person, task, person_assignments)
        print(f"  {person}: {'✓ Available' if available else '✗ Not available'}")
        
        if available and not assigned:
            # Assign the task to this person
            person_assignments[person].append(task)
            print(f"  → Assigned {task_name} to {person}")
            assignment_results.append((task_name, person))
            assigned = True
    
    if not assigned:
        print(f"  ⚠️  Could not assign {task_name} to anyone!")
        assignment_results.append((task_name, "UNASSIGNED"))

# Summary of assignments
print("\n" + "="*60)
print("FINAL ASSIGNMENT SUMMARY")
print("="*60)

for person, tasks in person_assignments.items():
    if tasks:
        print(f"\n👤 {person}:")
        for task in tasks:
            print(f"  📋 {task['Name']} ({task['Start']} - {task['Finish']})")
    else:
        print(f"\n👤 {person}: No assignments")

print("\n📊 Assignment workflow results:")
for task_name, assignee in assignment_results:
    status = "✓" if assignee != "UNASSIGNED" else "✗"
    print(f"  {status} {task_name}: {assignee}")

print(f"\n🎯 Person assignment system testing completed!")
print(f"   Total people: {len(person_assignments)}")
print(f"   Total assigned tasks: {sum(len(tasks) for tasks in person_assignments.values())}")
print(f"   Successfully assigned: {len([r for r in assignment_results if r[1] != 'UNASSIGNED'])}")
print(f"   Unassigned: {len([r for r in assignment_results if r[1] == 'UNASSIGNED'])}")


In [ ]:
# Testing the Main Assignment Method: assign_tasks_to_persons
# =========================================================

print("Testing assign_tasks_to_persons Method")
print("=" * 60)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

print(f"📊 Project: {project.name}")
print(f"📅 Start Date: {project.start_date}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# Display critical path information
critical_tasks = [task for task in scheduled_tasks if task.get('TF', float('inf')) == 0]
print(f"🔥 Critical Path Tasks: {len(critical_tasks)}")

print(f"\n🎯 Critical Path Tasks:")
for task in critical_tasks:
    print(f"  • {task['Name']} (TF: {task.get('TF', 'N/A')})")

# Use the resource mapping from the earlier cell
# Make sure resource name stems are properly defined for all resource types
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',        # Fixed typo from earlier
    'DBA': 'DBA',                   # Direct mapping for database architects
    'Quality Control Tester': 'QC',
    'Tester': 'Test'                # Alternative name for testers
}

print(f"\n📋 Resource Types Available:")
for resource_type, stem in resource_name_stems_updated.items():
    print(f"  {resource_type}: {stem}_X")

print(f"\n🔍 Task-Resource Mapping:")
task_count_by_resource = {}
for task_name, resource_type in who_does_what.items():
    if resource_type not in task_count_by_resource:
        task_count_by_resource[resource_type] = 0
    task_count_by_resource[resource_type] += 1

for resource_type, count in sorted(task_count_by_resource.items()):
    print(f"  {resource_type}: {count} task(s)")

print(f"\n" + "=" * 80)
print("🚀 EXECUTING ASSIGNMENT ALGORITHM")
print("=" * 80)

# Execute the main assignment method
try:
    person_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated
    )
    
    print(f"\n✅ Assignment completed successfully!")
    
except Exception as e:
    print(f"\n❌ Assignment failed: {e}")
    raise

print(f"\n" + "=" * 80)
print("📊 DETAILED ASSIGNMENT ANALYSIS")
print("=" * 80)

# Analyze the results
total_people = len(person_assignments)
total_task_assignments = sum(len(tasks) for tasks in person_assignments.values())

print(f"\n👥 People Created: {total_people}")
print(f"📋 Task Assignments: {total_task_assignments}")

# Group people by resource type for analysis
people_by_resource = {}
for person_name, assigned_tasks in person_assignments.items():
    # Extract resource type from person name (before the underscore)
    resource_stem = person_name.split('_')[0]
    
    # Find the full resource type name
    resource_type = None
    for full_type, stem in resource_name_stems_updated.items():
        if stem == resource_stem:
            resource_type = full_type
            break
    
    if resource_type is None:
        resource_type = resource_stem  # Fallback
    
    if resource_type not in people_by_resource:
        people_by_resource[resource_type] = []
    
    people_by_resource[resource_type].append((person_name, assigned_tasks))

print(f"\n👥 PEOPLE BY RESOURCE TYPE:")
print("-" * 40)
for resource_type, people_list in sorted(people_by_resource.items()):
    print(f"\n🔹 {resource_type} ({len(people_list)} people):")
    
    for person_name, assigned_tasks in people_list:
        critical_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
        total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
        
        print(f"  👤 {person_name}:")
        print(f"     Tasks: {len(assigned_tasks)} | Critical: {critical_count} | Duration: {total_duration} days")
        
        # Show first few tasks
        for i, task in enumerate(assigned_tasks[:3]):  # Show first 3 tasks
            critical_indicator = "🔥" if task.get('TF', float('inf')) == 0 else "  "
            print(f"     {critical_indicator} {task['Name']} ({task['Start']} - {task['Finish']})")
        
        if len(assigned_tasks) > 3:
            print(f"     ... and {len(assigned_tasks) - 3} more tasks")

print(f"\n" + "=" * 80)
print("⏱️  WORKLOAD ANALYSIS")
print("=" * 80)

# Calculate workload statistics
workload_stats = []
for person_name, assigned_tasks in person_assignments.items():
    total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
    critical_tasks_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
    
    # Calculate date range
    if assigned_tasks:
        start_dates = [task['Start'] for task in assigned_tasks]
        finish_dates = [task['Finish'] for task in assigned_tasks]
        earliest_start = min(start_dates)
        latest_finish = max(finish_dates)
        date_span = (latest_finish - earliest_start).days + 1
    else:
        date_span = 0
        earliest_start = latest_finish = None
    
    workload_stats.append({
        'person': person_name,
        'task_count': len(assigned_tasks),
        'total_duration': total_duration,
        'critical_tasks': critical_tasks_count,
        'date_span': date_span,
        'earliest_start': earliest_start,
        'latest_finish': latest_finish
    })

# Sort by total duration (heaviest workload first)
workload_stats.sort(key=lambda x: x['total_duration'], reverse=True)

print(f"\n📈 Top 10 People by Workload:")
print(f"{'Person':<15} {'Tasks':<6} {'Critical':<9} {'Duration':<9} {'Span':<8} {'Period'}")
print("-" * 80)

for i, stats in enumerate(workload_stats[:10]):
    period = f"{stats['earliest_start']} to {stats['latest_finish']}" if stats['earliest_start'] else "No tasks"
    print(f"{stats['person']:<15} {stats['task_count']:<6} {stats['critical_tasks']:<9} {stats['total_duration']:<9} {stats['date_span']:<8} {period}")

# Overall statistics
total_critical_assignments = sum(stats['critical_tasks'] for stats in workload_stats)
avg_tasks_per_person = total_task_assignments / total_people if total_people > 0 else 0
avg_duration_per_person = sum(stats['total_duration'] for stats in workload_stats) / total_people if total_people > 0 else 0

print(f"\n📊 SUMMARY STATISTICS:")
print(f"   Total critical path assignments: {total_critical_assignments}")  
print(f"   Average tasks per person: {avg_tasks_per_person:.1f}")
print(f"   Average duration per person: {avg_duration_per_person:.1f} days")

# Check for potential issues
overloaded_people = [stats for stats in workload_stats if stats['total_duration'] > 100]  # More than 100 days
underutilized_people = [stats for stats in workload_stats if stats['total_duration'] < 20]  # Less than 20 days

if overloaded_people:
    print(f"\n⚠️  Potentially overloaded people ({len(overloaded_people)}):")
    for stats in overloaded_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

if underutilized_people:
    print(f"\n💡 Potentially underutilized people ({len(underutilized_people)}):")
    for stats in underutilized_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

print(f"\n🎯 Assignment algorithm completed successfully!")
print(f"   Created {total_people} people across {len(people_by_resource)} resource types")
print(f"   All {len(scheduled_tasks)} tasks have been assigned")
print(f"   Critical path is properly staffed with {total_critical_assignments} assignments")


In [ ]:
# Testing the Main Assignment Method: assign_tasks_to_persons
# =========================================================

print("Testing assign_tasks_to_persons Method")
print("=" * 60)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

print(f"📊 Project: {project.name}")
print(f"📅 Start Date: {project.start_date}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# Display critical path information
critical_tasks = [task for task in scheduled_tasks if task.get('TF', float('inf')) == 0]
print(f"🔥 Critical Path Tasks: {len(critical_tasks)}")

print(f"\n🎯 Critical Path Tasks:")
for task in critical_tasks:
    print(f"  • {task['Name']} (TF: {task.get('TF', 'N/A')})")

# Use the resource mapping from the earlier cell
# Make sure resource name stems are properly defined for all resource types
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',        # Fixed typo from earlier
    'DBA': 'DBA',                   # Direct mapping for database architects
    'Quality Control Tester': 'QC',
    'Tester': 'Test'                # Alternative name for testers
}

print(f"\n📋 Resource Types Available:")
for resource_type, stem in resource_name_stems_updated.items():
    print(f"  {resource_type}: {stem}_X")

print(f"\n🔍 Task-Resource Mapping:")
task_count_by_resource = {}
for task_name, resource_type in who_does_what.items():
    if resource_type not in task_count_by_resource:
        task_count_by_resource[resource_type] = 0
    task_count_by_resource[resource_type] += 1

for resource_type, count in sorted(task_count_by_resource.items()):
    print(f"  {resource_type}: {count} task(s)")

print(f"\n" + "=" * 80)
print("🚀 EXECUTING ASSIGNMENT ALGORITHM")
print("=" * 80)

# Execute the main assignment method
try:
    person_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated
    )
    
    print(f"\n✅ Assignment completed successfully!")
    
except Exception as e:
    print(f"\n❌ Assignment failed: {e}")
    raise

print(f"\n" + "=" * 80)
print("📊 DETAILED ASSIGNMENT ANALYSIS")
print("=" * 80)

# Analyze the results
total_people = len(person_assignments)
total_task_assignments = sum(len(tasks) for tasks in person_assignments.values())

print(f"\n👥 People Created: {total_people}")
print(f"📋 Task Assignments: {total_task_assignments}")

# Group people by resource type for analysis
people_by_resource = {}
for person_name, assigned_tasks in person_assignments.items():
    # Extract resource type from person name (before the underscore)
    resource_stem = person_name.split('_')[0]
    
    # Find the full resource type name
    resource_type = None
    for full_type, stem in resource_name_stems_updated.items():
        if stem == resource_stem:
            resource_type = full_type
            break
    
    if resource_type is None:
        resource_type = resource_stem  # Fallback
    
    if resource_type not in people_by_resource:
        people_by_resource[resource_type] = []
    
    people_by_resource[resource_type].append((person_name, assigned_tasks))

print(f"\n👥 PEOPLE BY RESOURCE TYPE:")
print("-" * 40)
for resource_type, people_list in sorted(people_by_resource.items()):
    print(f"\n🔹 {resource_type} ({len(people_list)} people):")
    
    for person_name, assigned_tasks in people_list:
        critical_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
        total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
        
        print(f"  👤 {person_name}:")
        print(f"     Tasks: {len(assigned_tasks)} | Critical: {critical_count} | Duration: {total_duration} days")
        
        # Show first few tasks
        for i, task in enumerate(assigned_tasks[:3]):  # Show first 3 tasks
            critical_indicator = "🔥" if task.get('TF', float('inf')) == 0 else "  "
            print(f"     {critical_indicator} {task['Name']} ({task['Start']} - {task['Finish']})")
        
        if len(assigned_tasks) > 3:
            print(f"     ... and {len(assigned_tasks) - 3} more tasks")

print(f"\n" + "=" * 80)
print("⏱️  WORKLOAD ANALYSIS")
print("=" * 80)

# Calculate workload statistics
workload_stats = []
for person_name, assigned_tasks in person_assignments.items():
    total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
    critical_tasks_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
    
    # Calculate date range
    if assigned_tasks:
        start_dates = [task['Start'] for task in assigned_tasks]
        finish_dates = [task['Finish'] for task in assigned_tasks]
        earliest_start = min(start_dates)
        latest_finish = max(finish_dates)
        date_span = (latest_finish - earliest_start).days + 1
    else:
        date_span = 0
        earliest_start = latest_finish = None
    
    workload_stats.append({
        'person': person_name,
        'task_count': len(assigned_tasks),
        'total_duration': total_duration,
        'critical_tasks': critical_tasks_count,
        'date_span': date_span,
        'earliest_start': earliest_start,
        'latest_finish': latest_finish
    })

# Sort by total duration (heaviest workload first)
workload_stats.sort(key=lambda x: x['total_duration'], reverse=True)

print(f"\n📈 Top 10 People by Workload:")
print(f"{'Person':<15} {'Tasks':<6} {'Critical':<9} {'Duration':<9} {'Span':<8} {'Period'}")
print("-" * 80)

for i, stats in enumerate(workload_stats[:10]):
    period = f"{stats['earliest_start']} to {stats['latest_finish']}" if stats['earliest_start'] else "No tasks"
    print(f"{stats['person']:<15} {stats['task_count']:<6} {stats['critical_tasks']:<9} {stats['total_duration']:<9} {stats['date_span']:<8} {period}")

# Overall statistics
total_critical_assignments = sum(stats['critical_tasks'] for stats in workload_stats)
avg_tasks_per_person = total_task_assignments / total_people if total_people > 0 else 0
avg_duration_per_person = sum(stats['total_duration'] for stats in workload_stats) / total_people if total_people > 0 else 0

print(f"\n📊 SUMMARY STATISTICS:")
print(f"   Total critical path assignments: {total_critical_assignments}")  
print(f"   Average tasks per person: {avg_tasks_per_person:.1f}")
print(f"   Average duration per person: {avg_duration_per_person:.1f} days")

# Check for potential issues
overloaded_people = [stats for stats in workload_stats if stats['total_duration'] > 100]  # More than 100 days
underutilized_people = [stats for stats in workload_stats if stats['total_duration'] < 20]  # Less than 20 days

if overloaded_people:
    print(f"\n⚠️  Potentially overloaded people ({len(overloaded_people)}):")
    for stats in overloaded_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

if underutilized_people:
    print(f"\n💡 Potentially underutilized people ({len(underutilized_people)}):")
    for stats in underutilized_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

print(f"\n🎯 Assignment algorithm completed successfully!")
print(f"   Created {total_people} people across {len(people_by_resource)} resource types")
print(f"   All {len(scheduled_tasks)} tasks have been assigned")
print(f"   Critical path is properly staffed with {total_critical_assignments} assignments")


In [ ]:
# Testing the Main Assignment Method: assign_tasks_to_persons
# =========================================================

print("Testing assign_tasks_to_persons Method")
print("=" * 60)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

print(f"📊 Project: {project.name}")
print(f"📅 Start Date: {project.start_date}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# Display critical path information
critical_tasks = [task for task in scheduled_tasks if task.get('TF', float('inf')) == 0]
print(f"🔥 Critical Path Tasks: {len(critical_tasks)}")

print(f"\n🎯 Critical Path Tasks:")
for task in critical_tasks:
    print(f"  • {task['Name']} (TF: {task.get('TF', 'N/A')})")

# Use the resource mapping from the earlier cell
# Make sure resource name stems are properly defined for all resource types
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',        # Fixed typo from earlier
    'DBA': 'DBA',                   # Direct mapping for database architects
    'Quality Control Tester': 'QC',
    'Tester': 'Test'                # Alternative name for testers
}

print(f"\n📋 Resource Types Available:")
for resource_type, stem in resource_name_stems_updated.items():
    print(f"  {resource_type}: {stem}_X")

print(f"\n🔍 Task-Resource Mapping:")
task_count_by_resource = {}
for task_name, resource_type in who_does_what.items():
    if resource_type not in task_count_by_resource:
        task_count_by_resource[resource_type] = 0
    task_count_by_resource[resource_type] += 1

for resource_type, count in sorted(task_count_by_resource.items()):
    print(f"  {resource_type}: {count} task(s)")

print(f"\n" + "=" * 80)
print("🚀 EXECUTING ASSIGNMENT ALGORITHM")
print("=" * 80)

# Execute the main assignment method
try:
    person_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated
    )
    
    print(f"\n✅ Assignment completed successfully!")
    
except Exception as e:
    print(f"\n❌ Assignment failed: {e}")
    raise

print(f"\n" + "=" * 80)
print("📊 DETAILED ASSIGNMENT ANALYSIS")
print("=" * 80)

# Analyze the results
total_people = len(person_assignments)
total_task_assignments = sum(len(tasks) for tasks in person_assignments.values())

print(f"\n👥 People Created: {total_people}")
print(f"📋 Task Assignments: {total_task_assignments}")

# Group people by resource type for analysis
people_by_resource = {}
for person_name, assigned_tasks in person_assignments.items():
    # Extract resource type from person name (before the underscore)
    resource_stem = person_name.split('_')[0]
    
    # Find the full resource type name
    resource_type = None
    for full_type, stem in resource_name_stems_updated.items():
        if stem == resource_stem:
            resource_type = full_type
            break
    
    if resource_type is None:
        resource_type = resource_stem  # Fallback
    
    if resource_type not in people_by_resource:
        people_by_resource[resource_type] = []
    
    people_by_resource[resource_type].append((person_name, assigned_tasks))

print(f"\n👥 PEOPLE BY RESOURCE TYPE:")
print("-" * 40)
for resource_type, people_list in sorted(people_by_resource.items()):
    print(f"\n🔹 {resource_type} ({len(people_list)} people):")
    
    for person_name, assigned_tasks in people_list:
        critical_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
        total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
        
        print(f"  👤 {person_name}:")
        print(f"     Tasks: {len(assigned_tasks)} | Critical: {critical_count} | Duration: {total_duration} days")
        
        # Show first few tasks
        for i, task in enumerate(assigned_tasks[:3]):  # Show first 3 tasks
            critical_indicator = "🔥" if task.get('TF', float('inf')) == 0 else "  "
            print(f"     {critical_indicator} {task['Name']} ({task['Start']} - {task['Finish']})")
        
        if len(assigned_tasks) > 3:
            print(f"     ... and {len(assigned_tasks) - 3} more tasks")

print(f"\n" + "=" * 80)
print("⏱️  WORKLOAD ANALYSIS")
print("=" * 80)

# Calculate workload statistics
workload_stats = []
for person_name, assigned_tasks in person_assignments.items():
    total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
    critical_tasks_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
    
    # Calculate date range
    if assigned_tasks:
        start_dates = [task['Start'] for task in assigned_tasks]
        finish_dates = [task['Finish'] for task in assigned_tasks]
        earliest_start = min(start_dates)
        latest_finish = max(finish_dates)
        date_span = (latest_finish - earliest_start).days + 1
    else:
        date_span = 0
        earliest_start = latest_finish = None
    
    workload_stats.append({
        'person': person_name,
        'task_count': len(assigned_tasks),
        'total_duration': total_duration,
        'critical_tasks': critical_tasks_count,
        'date_span': date_span,
        'earliest_start': earliest_start,
        'latest_finish': latest_finish
    })

# Sort by total duration (heaviest workload first)
workload_stats.sort(key=lambda x: x['total_duration'], reverse=True)

print(f"\n📈 Top 10 People by Workload:")
print(f"{'Person':<15} {'Tasks':<6} {'Critical':<9} {'Duration':<9} {'Span':<8} {'Period'}")
print("-" * 80)

for i, stats in enumerate(workload_stats[:10]):
    period = f"{stats['earliest_start']} to {stats['latest_finish']}" if stats['earliest_start'] else "No tasks"
    print(f"{stats['person']:<15} {stats['task_count']:<6} {stats['critical_tasks']:<9} {stats['total_duration']:<9} {stats['date_span']:<8} {period}")

# Overall statistics
total_critical_assignments = sum(stats['critical_tasks'] for stats in workload_stats)
avg_tasks_per_person = total_task_assignments / total_people if total_people > 0 else 0
avg_duration_per_person = sum(stats['total_duration'] for stats in workload_stats) / total_people if total_people > 0 else 0

print(f"\n📊 SUMMARY STATISTICS:")
print(f"   Total critical path assignments: {total_critical_assignments}")  
print(f"   Average tasks per person: {avg_tasks_per_person:.1f}")
print(f"   Average duration per person: {avg_duration_per_person:.1f} days")

# Check for potential issues
overloaded_people = [stats for stats in workload_stats if stats['total_duration'] > 100]  # More than 100 days
underutilized_people = [stats for stats in workload_stats if stats['total_duration'] < 20]  # Less than 20 days

if overloaded_people:
    print(f"\n⚠️  Potentially overloaded people ({len(overloaded_people)}):")
    for stats in overloaded_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

if underutilized_people:
    print(f"\n💡 Potentially underutilized people ({len(underutilized_people)}):")
    for stats in underutilized_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

print(f"\n🎯 Assignment algorithm completed successfully!")
print(f"   Created {total_people} people across {len(people_by_resource)} resource types")
print(f"   All {len(scheduled_tasks)} tasks have been assigned")
print(f"   Critical path is properly staffed with {total_critical_assignments} assignments")


In [ ]:
# Testing the Main Assignment Method: assign_tasks_to_persons
# =========================================================

print("Testing assign_tasks_to_persons Method")
print("=" * 60)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

print(f"📊 Project: {project.name}")
print(f"📅 Start Date: {project.start_date}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# Display critical path information
critical_tasks = [task for task in scheduled_tasks if task.get('TF', float('inf')) == 0]
print(f"🔥 Critical Path Tasks: {len(critical_tasks)}")

print(f"\n🎯 Critical Path Tasks:")
for task in critical_tasks:
    print(f"  • {task['Name']} (TF: {task.get('TF', 'N/A')})")

# Use the resource mapping from the earlier cell
# Make sure resource name stems are properly defined for all resource types
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',        # Fixed typo from earlier
    'DBA': 'DBA',                   # Direct mapping for database architects
    'Quality Control Tester': 'QC',
    'Tester': 'Test'                # Alternative name for testers
}

print(f"\n📋 Resource Types Available:")
for resource_type, stem in resource_name_stems_updated.items():
    print(f"  {resource_type}: {stem}_X")

print(f"\n🔍 Task-Resource Mapping:")
task_count_by_resource = {}
for task_name, resource_type in who_does_what.items():
    if resource_type not in task_count_by_resource:
        task_count_by_resource[resource_type] = 0
    task_count_by_resource[resource_type] += 1

for resource_type, count in sorted(task_count_by_resource.items()):
    print(f"  {resource_type}: {count} task(s)")

print(f"\n" + "=" * 80)
print("🚀 EXECUTING ASSIGNMENT ALGORITHM")
print("=" * 80)

# Execute the main assignment method
try:
    person_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated
    )
    
    print(f"\n✅ Assignment completed successfully!")
    
except Exception as e:
    print(f"\n❌ Assignment failed: {e}")
    raise

print(f"\n" + "=" * 80)
print("📊 DETAILED ASSIGNMENT ANALYSIS")
print("=" * 80)

# Analyze the results
total_people = len(person_assignments)
total_task_assignments = sum(len(tasks) for tasks in person_assignments.values())

print(f"\n👥 People Created: {total_people}")
print(f"📋 Task Assignments: {total_task_assignments}")

# Group people by resource type for analysis
people_by_resource = {}
for person_name, assigned_tasks in person_assignments.items():
    # Extract resource type from person name (before the underscore)
    resource_stem = person_name.split('_')[0]
    
    # Find the full resource type name
    resource_type = None
    for full_type, stem in resource_name_stems_updated.items():
        if stem == resource_stem:
            resource_type = full_type
            break
    
    if resource_type is None:
        resource_type = resource_stem  # Fallback
    
    if resource_type not in people_by_resource:
        people_by_resource[resource_type] = []
    
    people_by_resource[resource_type].append((person_name, assigned_tasks))

print(f"\n👥 PEOPLE BY RESOURCE TYPE:")
print("-" * 40)
for resource_type, people_list in sorted(people_by_resource.items()):
    print(f"\n🔹 {resource_type} ({len(people_list)} people):")
    
    for person_name, assigned_tasks in people_list:
        critical_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
        total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
        
        print(f"  👤 {person_name}:")
        print(f"     Tasks: {len(assigned_tasks)} | Critical: {critical_count} | Duration: {total_duration} days")
        
        # Show first few tasks
        for i, task in enumerate(assigned_tasks[:3]):  # Show first 3 tasks
            critical_indicator = "🔥" if task.get('TF', float('inf')) == 0 else "  "
            print(f"     {critical_indicator} {task['Name']} ({task['Start']} - {task['Finish']})")
        
        if len(assigned_tasks) > 3:
            print(f"     ... and {len(assigned_tasks) - 3} more tasks")

print(f"\n" + "=" * 80)
print("⏱️  WORKLOAD ANALYSIS")
print("=" * 80)

# Calculate workload statistics
workload_stats = []
for person_name, assigned_tasks in person_assignments.items():
    total_duration = sum(task.get('Duration', 0) for task in assigned_tasks)
    critical_tasks_count = sum(1 for task in assigned_tasks if task.get('TF', float('inf')) == 0)
    
    # Calculate date range
    if assigned_tasks:
        start_dates = [task['Start'] for task in assigned_tasks]
        finish_dates = [task['Finish'] for task in assigned_tasks]
        earliest_start = min(start_dates)
        latest_finish = max(finish_dates)
        date_span = (latest_finish - earliest_start).days + 1
    else:
        date_span = 0
        earliest_start = latest_finish = None
    
    workload_stats.append({
        'person': person_name,
        'task_count': len(assigned_tasks),
        'total_duration': total_duration,
        'critical_tasks': critical_tasks_count,
        'date_span': date_span,
        'earliest_start': earliest_start,
        'latest_finish': latest_finish
    })

# Sort by total duration (heaviest workload first)
workload_stats.sort(key=lambda x: x['total_duration'], reverse=True)

print(f"\n📈 Top 10 People by Workload:")
print(f"{'Person':<15} {'Tasks':<6} {'Critical':<9} {'Duration':<9} {'Span':<8} {'Period'}")
print("-" * 80)

for i, stats in enumerate(workload_stats[:10]):
    period = f"{stats['earliest_start']} to {stats['latest_finish']}" if stats['earliest_start'] else "No tasks"
    print(f"{stats['person']:<15} {stats['task_count']:<6} {stats['critical_tasks']:<9} {stats['total_duration']:<9} {stats['date_span']:<8} {period}")

# Overall statistics
total_critical_assignments = sum(stats['critical_tasks'] for stats in workload_stats)
avg_tasks_per_person = total_task_assignments / total_people if total_people > 0 else 0
avg_duration_per_person = sum(stats['total_duration'] for stats in workload_stats) / total_people if total_people > 0 else 0

print(f"\n📊 SUMMARY STATISTICS:")
print(f"   Total critical path assignments: {total_critical_assignments}")  
print(f"   Average tasks per person: {avg_tasks_per_person:.1f}")
print(f"   Average duration per person: {avg_duration_per_person:.1f} days")

# Check for potential issues
overloaded_people = [stats for stats in workload_stats if stats['total_duration'] > 100]  # More than 100 days
underutilized_people = [stats for stats in workload_stats if stats['total_duration'] < 20]  # Less than 20 days

if overloaded_people:
    print(f"\n⚠️  Potentially overloaded people ({len(overloaded_people)}):")
    for stats in overloaded_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

if underutilized_people:
    print(f"\n💡 Potentially underutilized people ({len(underutilized_people)}):")
    for stats in underutilized_people:
        print(f"   • {stats['person']}: {stats['total_duration']} days across {stats['task_count']} tasks")

print(f"\n🎯 Assignment algorithm completed successfully!")
print(f"   Created {total_people} people across {len(people_by_resource)} resource types")
print(f"   All {len(scheduled_tasks)} tasks have been assigned")
print(f"   Critical path is properly staffed with {total_critical_assignments} assignments")


In [ ]:
# Testing Assignment Validation: validate_assignments method
# ========================================================

print("Testing validate_assignments Method")
print("=" * 50)

# Get the scheduled task data from the project
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

# Resource mapping
resource_name_stems_updated = {
    'Architect': 'Arch',
    'Project Manager': 'PM', 
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',
    'DBA': 'DBA',
    'Quality Control Tester': 'QC',
    'Tester': 'Test'
}

print(f"📊 Project: {project.name}")
print(f"📋 Total Tasks: {len(scheduled_tasks)}")

# SCENARIO 1: Test with VALID assignments (from our main algorithm)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 1: Testing with VALID assignments")
print("=" * 80)

print("Running assignment algorithm to get valid assignments...")
try:
    valid_assignments = project.assign_tasks_to_persons(
        tasks=scheduled_tasks,
        task_resource_mapping=who_does_what,
        resource_name_stems=resource_name_stems_updated,
        max_gap_days=7
    )
    
    print(f"\n✅ Assignment algorithm completed successfully!")
    print(f"Now validating these assignments...")
    
    # Validate the assignments
    is_valid = project.validate_assignments(valid_assignments, scheduled_tasks)
    
    print(f"\n🎯 SCENARIO 1 RESULT: {'✅ PASSED' if is_valid else '❌ FAILED'}")
    
except Exception as e:
    print(f"❌ Assignment algorithm failed: {e}")
    valid_assignments = {}

# SCENARIO 2: Test with INVALID assignments (intentional overlaps)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 2: Testing with INVALID assignments (overlapping tasks)")
print("=" * 80)

print("Creating intentionally invalid assignments with overlapping tasks...")

# Helper function to find a task by name
def find_task_by_name(task_name):
    for task in scheduled_tasks:
        if task['Name'] == task_name:
            return task
    return None

# Create invalid assignments with overlapping tasks
invalid_assignments = {}

# Assign overlapping tasks to same person (this should fail validation)
developer_1_tasks = []
logging_task = find_task_by_name('Logging')
security_task = find_task_by_name('Security')
pubsub_task = find_task_by_name('Pub/Sub')

if logging_task and security_task and pubsub_task:
    # These tasks likely overlap (they all depend on task 3)
    developer_1_tasks = [logging_task, security_task, pubsub_task]
    invalid_assignments['Dev_1'] = developer_1_tasks
    
    print(f"Assigned overlapping tasks to Dev_1:")
    for task in developer_1_tasks:
        print(f"  • {task['Name']} ({task['Start']} - {task['Finish']})")

# Add some other assignments to make it more realistic
architect_task = find_task_by_name('Requirements')
if architect_task:
    invalid_assignments['Arch_1'] = [architect_task]

# Validate the invalid assignments
print(f"\nNow validating these INTENTIONALLY INVALID assignments...")
is_valid_2 = project.validate_assignments(invalid_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 2 RESULT: {'❌ FAILED as expected' if not is_valid_2 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 3: Test with INCOMPLETE assignments (missing tasks)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 3: Testing with INCOMPLETE assignments (missing tasks)")
print("=" * 80)

print("Creating incomplete assignments (not all tasks assigned)...")

# Create assignments with only some tasks
incomplete_assignments = {}

# Only assign first few tasks
first_few_tasks = scheduled_tasks[:5]  # Only first 5 tasks
for i, task in enumerate(first_few_tasks):
    person_name = f"Person_{i+1}"
    incomplete_assignments[person_name] = [task]
    print(f"  • {person_name}: {task['Name']}")

print(f"\nOnly assigned {len(first_few_tasks)} out of {len(scheduled_tasks)} tasks")

# Validate the incomplete assignments  
print(f"\nNow validating these INCOMPLETE assignments...")
is_valid_3 = project.validate_assignments(incomplete_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 3 RESULT: {'❌ FAILED as expected' if not is_valid_3 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 4: Test with DUPLICATE assignments (same task assigned twice)
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 4: Testing with DUPLICATE assignments (same task assigned twice)")
print("=" * 80)

print("Creating assignments with duplicate task assignments...")

# Create assignments where same task is assigned to multiple people
duplicate_assignments = {}

requirements_task = find_task_by_name('Requirements')
if requirements_task:
    # Assign same task to two different people
    duplicate_assignments['Person_A'] = [requirements_task]
    duplicate_assignments['Person_B'] = [requirements_task]  # Same task!
    
    print(f"Assigned '{requirements_task['Name']}' to both Person_A and Person_B")

# Add a few more normal assignments
for i, task in enumerate(scheduled_tasks[1:4]):  # Tasks 2, 3, 4
    person_name = f"Person_{i+3}"
    duplicate_assignments[person_name] = [task]

# Validate the duplicate assignments
print(f"\nNow validating these assignments with DUPLICATES...")
is_valid_4 = project.validate_assignments(duplicate_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 4 RESULT: {'❌ FAILED as expected' if not is_valid_4 else '⚠️ UNEXPECTEDLY PASSED'}")

# SCENARIO 5: Test with EMPTY assignments
print(f"\n" + "=" * 80)
print("🧪 SCENARIO 5: Testing with EMPTY assignments")
print("=" * 80)

print("Testing validation with empty assignments...")

empty_assignments = {}

# Validate empty assignments
is_valid_5 = project.validate_assignments(empty_assignments, scheduled_tasks)

print(f"\n🎯 SCENARIO 5 RESULT: {'❌ FAILED as expected' if not is_valid_5 else '⚠️ UNEXPECTEDLY PASSED'}")

# SUMMARY OF ALL TEST SCENARIOS
print(f"\n" + "=" * 80)
print("🎯 VALIDATION TESTING SUMMARY")
print("=" * 80)

scenarios = [
    ("VALID assignments", is_valid if 'is_valid' in locals() else False, True),
    ("OVERLAPPING assignments", is_valid_2, False),
    ("INCOMPLETE assignments", is_valid_3, False),
    ("DUPLICATE assignments", is_valid_4, False),
    ("EMPTY assignments", is_valid_5, False)
]

print(f"\n📊 Test Results:")
print(f"{'Scenario':<25} {'Result':<10} {'Expected':<10} {'Status'}")
print("-" * 60)

all_tests_passed = True

for i, (scenario, actual, expected) in enumerate(scenarios, 1):
    status = "✅ PASS" if actual == expected else "❌ FAIL"
    if actual != expected:
        all_tests_passed = False
    
    actual_str = "VALID" if actual else "INVALID"
    expected_str = "VALID" if expected else "INVALID"
    
    print(f"{scenario:<25} {actual_str:<10} {expected_str:<10} {status}")

print(f"\n🎯 OVERALL VALIDATION TESTING: {'✅ ALL TESTS PASSED' if all_tests_passed else '❌ SOME TESTS FAILED'}")

if all_tests_passed:
    print("🎉 The validate_assignments method correctly identifies:")
    print("   • Valid assignments with no overlaps")
    print("   • Invalid assignments with overlapping tasks")
    print("   • Incomplete assignments with missing tasks")
    print("   • Invalid assignments with duplicate tasks")
    print("   • Empty assignments")
    print("\n✨ Assignment validation is working perfectly!")
else:
    print("⚠️  Some validation tests failed. Check the implementation.")

print("\n📝 The validation method provides:")
print("   • Detailed overlap checking per person")
print("   • Complete task assignment verification")
print("   • Comprehensive assignment summaries")
print("   • Clear pass/fail validation results")
print("   • Detailed error reporting and warnings")
print("   • Workload balance analysis")
